In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, functions

In [2]:
conf = SparkConf().set("spark.jars", "/Users/rich/Desktop/project/jars/spark-excel_2.12-3.5.0_0.20.3.jar")

In [3]:
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

24/03/22 14:23:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/22 14:23:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.format('excel').option("maxRowsInMemory", 20).load("/Users/rich/Desktop/project/Data/Raw Data_To interns_Contacts.xlsx")

In [8]:
df.show()

24/03/22 14:23:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----------+--------------+---------+---------------+----------+-----+-----------+---------------+------------+-----------------+--------------+----------+--------------------+-------------+--------------+----------------------+-----------+------+-------------+------+-----------+--------------------+------------------+----------------+--------------+---------------------+------------+------------+---------------+-------+----------------+-------------+
|  Id|externalid|MasterRecordId|AccountId|IsPersonAccount|Salutation| Name|MailingCity|MailingDistrict|MailingState|MailingPostalCode|MailingCountry|    Mobile|               Email|    Birthdate|   CreatedDate|mobile_country_code__c|nickname__c|sex__c|crm_custid__c| no__c| fb_uuid__c|        line_uuid__c|member_level_id__c|register_type__c| login_date__c|last_transfer_date__c|is_epaper__c|is_useful__c|block_status__c|site__c|register_date__c|modify_day__c|
+----+----------+--------------+---------+---------------+----------+-----+---

In [5]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- externalid: string (nullable = true)
 |-- MasterRecordId: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- IsPersonAccount: string (nullable = true)
 |-- Salutation: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- MailingCity: string (nullable = true)
 |-- MailingDistrict: string (nullable = true)
 |-- MailingState: string (nullable = true)
 |-- MailingPostalCode: string (nullable = true)
 |-- MailingCountry: string (nullable = true)
 |-- Mobile: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- CreatedDate: string (nullable = true)
 |-- mobile_country_code__c: string (nullable = true)
 |-- nickname__c: string (nullable = true)
 |-- sex__c: string (nullable = true)
 |-- crm_custid__c: string (nullable = true)
 |-- no__c: string (nullable = true)
 |-- fb_uuid__c: string (nullable = true)
 |-- line_uuid__c: string (nullable = true)
 |-- member_leve

In [63]:
df.count()

24/03/15 18:04:09 WARN ExcelHeaderChecker: Number of column in Excel header is not equal to number of fields in the schema:
 Header length: 33, schema size: 0
Excel file: file:///Users/dragonh/Workspace/intern/2024Q2/raw_data/Raw%20Data_To%20interns_Contacts.xlsx


67871

In [64]:
df.show()

+----+----------+--------------+---------+---------------+----------+-----+-----------+---------------+------------+-----------------+--------------+----------+--------------------+-------------+--------------+----------------------+-----------+------+-------------+------+-----------+--------------------+------------------+----------------+--------------+---------------------+------------+------------+---------------+-------+----------------+-------------+
|  Id|externalid|MasterRecordId|AccountId|IsPersonAccount|Salutation| Name|MailingCity|MailingDistrict|MailingState|MailingPostalCode|MailingCountry|    Mobile|               Email|    Birthdate|   CreatedDate|mobile_country_code__c|nickname__c|sex__c|crm_custid__c| no__c| fb_uuid__c|        line_uuid__c|member_level_id__c|register_type__c| login_date__c|last_transfer_date__c|is_epaper__c|is_useful__c|block_status__c|site__c|register_date__c|modify_day__c|
+----+----------+--------------+---------+---------------+----------+-----+---

In [65]:
# df.select('MailingCity').pandas_api().plot.pie(y='MailingCity')
df.groupBy('MailingCity').count().select('MailingCity', functions.col('count')).pandas_api().plot.pie(y='count')

24/03/15 18:04:22 WARN ExcelHeaderChecker: Number of column in Excel header is not equal to number of fields in the schema:
 Header length: 33, schema size: 1
Excel file: file:///Users/dragonh/Workspace/intern/2024Q2/raw_data/Raw%20Data_To%20interns_Contacts.xlsx


In [66]:
df.createOrReplaceTempView('contact')

In [67]:
spark.sql(
    """
        SELECT
            *
        FROM
            contact
    """
).show()

+----+----------+--------------+---------+---------------+----------+-----+-----------+---------------+------------+-----------------+--------------+----------+--------------------+-------------+--------------+----------------------+-----------+------+-------------+------+-----------+--------------------+------------------+----------------+--------------+---------------------+------------+------------+---------------+-------+----------------+-------------+
|  Id|externalid|MasterRecordId|AccountId|IsPersonAccount|Salutation| Name|MailingCity|MailingDistrict|MailingState|MailingPostalCode|MailingCountry|    Mobile|               Email|    Birthdate|   CreatedDate|mobile_country_code__c|nickname__c|sex__c|crm_custid__c| no__c| fb_uuid__c|        line_uuid__c|member_level_id__c|register_type__c| login_date__c|last_transfer_date__c|is_epaper__c|is_useful__c|block_status__c|site__c|register_date__c|modify_day__c|
+----+----------+--------------+---------+---------------+----------+-----+---

In [68]:
member_create_analysis_df = spark.sql(
    """
        SELECT
            contact_formated.created_month as created_month,
            COUNT(1) AS created_count
        FROM (
            SELECT
                DATE_TRUNC('MM', to_timestamp(CreatedDate, 'M/d/yy HH:mm')) AS created_month
            FROM
                contact
        ) contact_formated
        GROUP BY
            created_month
        ORDER BY
            created_month
        ASC
    """
)
member_create_analysis_df.show()

24/03/15 18:04:35 WARN ExcelHeaderChecker: Number of column in Excel header is not equal to number of fields in the schema:
 Header length: 33, schema size: 1
Excel file: file:///Users/dragonh/Workspace/intern/2024Q2/raw_data/Raw%20Data_To%20interns_Contacts.xlsx


+-------------------+-------------+
|      created_month|created_count|
+-------------------+-------------+
|2020-03-01 00:00:00|        43030|
|2020-04-01 00:00:00|         1489|
|2020-05-01 00:00:00|         1084|
|2020-06-01 00:00:00|         1317|
|2020-07-01 00:00:00|         1885|
|2020-08-01 00:00:00|         2476|
|2020-09-01 00:00:00|         1205|
|2020-10-01 00:00:00|         1241|
|2020-11-01 00:00:00|         2785|
|2020-12-01 00:00:00|         1210|
|2021-01-01 00:00:00|          885|
|2021-02-01 00:00:00|          756|
|2021-03-01 00:00:00|          819|
|2021-04-01 00:00:00|          640|
|2021-05-01 00:00:00|          944|
|2021-06-01 00:00:00|         1980|
|2021-07-01 00:00:00|         2486|
|2021-08-01 00:00:00|         1409|
|2021-09-01 00:00:00|          210|
|2021-10-01 00:00:00|            1|
+-------------------+-------------+
only showing top 20 rows



In [69]:
member_create_analysis_df.pandas_api().plot.line(x='created_month', y='created_count')

24/03/15 18:04:47 WARN ExcelHeaderChecker: Number of column in Excel header is not equal to number of fields in the schema:
 Header length: 33, schema size: 1
Excel file: file:///Users/dragonh/Workspace/intern/2024Q2/raw_data/Raw%20Data_To%20interns_Contacts.xlsx
24/03/15 18:04:59 WARN ExcelHeaderChecker: Number of column in Excel header is not equal to number of fields in the schema:
 Header length: 33, schema size: 1
Excel file: file:///Users/dragonh/Workspace/intern/2024Q2/raw_data/Raw%20Data_To%20interns_Contacts.xlsx


In [ ]:
member_create_analysis_df.write.format('parquet').option('path', 's3:xxxxx').saveAsTable('database.tablename')